In [25]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

In [26]:
# 开启gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) # cuda

cuda


In [27]:
# 载入数据集
dataset = FashionMNIST(
    root='data',
    train=True,
    transform=transforms.ToTensor(),
    download=True)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=64,
    shuffle=True)

In [28]:
# 定义变分自编码器
class VAE(nn.Module):
    def __init__(self, image_size=784):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc2 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(400, 20)
        self.fc4 = nn.Linear(20, 400)
        self.fc5 = nn.Linear(400, 784)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var


vaeModel = VAE().to(device)
optimizer = torch.optim.Adam(vaeModel.parameters(), lr=1e-4)


In [29]:
for epoch in range(20):
    for i, (x, _) in enumerate(data_loader):

        x = x.to(device).view(-1, 784)
        x_reconst, mu, log_var = vaeModel(x)
        
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print("Epoch[{}/{}],Reconst Loss: {:.4f}".format(epoch+1, 20,reconst_loss.item()))
    
    with torch.no_grad():
        out, _, _ = vaeModel(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join('imagesAfterVAE', 'new-{}.png'.format(epoch+1)))


Epoch[1/20], Step [100/938], Reconst Loss: 26311.5723
Epoch[1/20], Step [200/938], Reconst Loss: 20754.8359
Epoch[1/20], Step [300/938], Reconst Loss: 19318.3438
Epoch[1/20], Step [400/938], Reconst Loss: 19580.2500
Epoch[1/20], Step [500/938], Reconst Loss: 18226.3809
Epoch[1/20], Step [600/938], Reconst Loss: 18728.6445
Epoch[1/20], Step [700/938], Reconst Loss: 19135.7773
Epoch[1/20], Step [800/938], Reconst Loss: 17596.9258
Epoch[1/20], Step [900/938], Reconst Loss: 17171.3770
Epoch[2/20], Step [100/938], Reconst Loss: 16697.4844
Epoch[2/20], Step [200/938], Reconst Loss: 17668.3398
Epoch[2/20], Step [300/938], Reconst Loss: 17061.6836
Epoch[2/20], Step [400/938], Reconst Loss: 17027.2617
Epoch[2/20], Step [500/938], Reconst Loss: 16153.0156
Epoch[2/20], Step [600/938], Reconst Loss: 15809.0986
Epoch[2/20], Step [700/938], Reconst Loss: 17049.0430
Epoch[2/20], Step [800/938], Reconst Loss: 16482.5586
Epoch[2/20], Step [900/938], Reconst Loss: 16534.3945
Epoch[3/20], Step [100/938],